In [1]:
#pip install nbconvert

In [2]:
#############################################################
####   CO2 emissions project first data analysis 
####     Complementary analysis 
####            BOXPLOTs Presentations
#############################################################

# Please, not you will need to run first the Starting point File. 

%run "000 - CO2 Starting Point.ipynb"  


In [3]:
# Review the distribution of  WLTP and NDEC CO2 indicators
# Prepare the data for the violin plot
# Melt the DataFrame to long-format for seaborn compatibility
melted_df = df_Ptred.melt(value_vars=['CO2_wltp', 'Enedc (g/km)'], var_name='Metric', value_name='Value')

# Create the violin plot
plt.figure(figsize=(6, 15))
violin = sns.violinplot(x='Metric', y='Value', data=melted_df, split=True, inner=None, palette='muted')

# Calculate and annotate metrics for each distribution
for i, metric in enumerate(['CO2_wltp', 'Enedc (g/km)']):
    # Extract the data for the current metric
    data = df_Ptred[metric].dropna()
    
    # Calculate metrics
    mean = data.mean()
    median = data.median()
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    
    # Annotate the violin plot with these metrics
    plt.text(i, data.min(), f'Min: {data.min():.2f}', ha='center')
    plt.text(i, q1, f'Q1: {q1:.2f}', ha='center')
    plt.text(i, median, f'Median: {median:.2f}', ha='center')
    plt.text(i, q3, f'Q3: {q3:.2f}', ha='center')
    plt.text(i, data.max(), f'Max: {data.max():.2f}', ha='center')
    plt.text(i, mean, f'Mean: {mean:.2f}', va='center', ha='center', color='white')

# Finalize the plot
plt.title('Distribution Comparison of CO2_wltp and Enedc (g/km)')
plt.ylabel('Values')
plt.xticks(np.arange(2), ['CO2_wltp', 'Enedc (g/km)'])

plt.show()

NameError: name 'df_Ptred' is not defined

In [ ]:
## Creating new df without NDEC column, removing new columns
df_ptwltp = df_Ptred.drop(['Enedc (g/km)', 'CO2_combined', 'WLTP_1_NEDC_0'], axis = 1)

#Removing all NaN values in 'CO2_wltp'
df_ptwltp.dropna(axis=0, subset=['CO2_wltp'], inplace=True)

df_ptwltp.head()
df_ptwltp.info()




                   

In [ ]:
# Averaging NaNs and 0 where the NaN % is small
variables_to_fill = ['Wheelbase_mm', 'Axle_width_steer_mm', 'Axle_width_other_mm', 'Power_KW', 'Test_mass']

# Ensure 'Veh_Model' and variables are in the correct format
df_ptwltp['Veh_Model'] = df_ptwltp['Veh_Model'].astype(str)

# Loop through the variables list to fill NaN values with the mean of the specific version
for var in variables_to_fill:
    # Group by 'Version' and calculate mean for the variable, excluding NaN 'Version' groups
    mean_values = df_ptwltp[df_ptwltp['Veh_Model'].notna()].groupby('Veh_Model')[var].transform('mean')
    
    # Fill NaN values in the variable with the corresponding mean of the 'Version'
    df_ptwltp[var] = df_ptwltp[var].fillna(mean_values)



### it has reduced NaNs a bit i will drop few 127 vehicles noe havinG Axle and Wheelbase and test mass

In [ ]:
#Removing all NaN values in 'CO2_wltp'
df_ptwltp.dropna(axis=0, subset=['Wheelbase_mm','Test_mass','Axle_width_steer_mm','Axle_width_other_mm'], inplace=True)

# Note: This approach assumes 'Version' NaNs should not be filled. If 'Version' itself has NaNs,
# calculations and fill operations are performed only on non-NaN 'Version' rows.

### checking NAN %
total_df_ptwltp = len(df_ptwltp)
nans_df_ptwltp = df_ptwltp.isna().sum()
percent_nans_df_ptwltp = (df_ptwltp.isna().sum() / total_records) * 100
nans_ptwltp_df = pd.DataFrame({'Nans': nans_df_ptwltp, 'Percentage of Total Nans (%)': percent_nans_df_ptwltp})

### Erwltp (g/km) zeroing nans
df_ptwltp['Erwltp (g/km)'] = df_ptwltp['Erwltp (g/km)'].fillna(0)

# Display the results
display("Nulls and their percentages:", nans_ptwltp_df)

df_ptwltp.duplicated().sum()


In [ ]:


# to view the final null and nan breakdown

# List of columns to analyze for NaN breakdown by 'Ft' (fuel type)
columns_to_analyze = ['Engine_cm3', 'Power_KW', 'El_Consumpt_whkm', 'Fuel consumption', 'Electric range (km)']

# Setting up the figure for subplots
n_rows = len(columns_to_analyze)
fig, axs = plt.subplots(n_rows, 1, figsize=(10, 5 * n_rows))

# Loop through each column and create a subplot
for i, column in enumerate(columns_to_analyze):
    # Calculate the percentage of NaN values for each 'Ft'
    nan_bd_ptwltp = df_ptwltp.groupby('Energy')[column].apply(lambda x: x.isna().mean() * 100)
    
    # Sort the breakdown for better visualization
    nan_bd_ptwltp_sorted = nan_bd_ptwltp.sort_values(ascending=False)
    
    # Plotting the breakdown
    sns.barplot(x= nan_bd_ptwltp_sorted.values, y= nan_bd_ptwltp_sorted.index, ax=axs[i], palette="viridis")
    axs[i].set_title(f'NaN Proportion in {column} by Fuel Type')
    axs[i].set_xlabel('Percentage of NaN Values')
    axs[i].set_ylabel('Fuel Type')

# Adjust layout to prevent overlap and ensure everything is visible
plt.tight_layout()
plt.show()



In [ ]:
## Correlation matrix creation

numerical_cols = ['CO2_wltp', 'Fuel consumption', 'Electric range (km)', 'Kg_veh', 'Test_mass', 'Power_KW', 'year', 'Eco-innovation program',  'Wheelbase_mm', 
                  'Axle_width_steer_mm', 'Axle_width_other_mm', 'Engine_cm3', 
                  'El_Consumpt_whkm', 'Erwltp (g/km)']
numerical_df = df_ptwltp[numerical_cols]

# Compute the correlation matrix and round to two decimal places
corr_matrix = numerical_df.corr().round(2)

# Drop any rows or columns that are all NaN, if any
corr_matrix_cleaned = corr_matrix.dropna(axis='columns', how='all').dropna(axis='rows', how='all')

display(corr_matrix_cleaned)

# Initialize matplotlib figure for size, make it larger if needed
plt.figure(figsize=(18, 12))

# Set font scale for annotations
sns.set(font_scale=1)

# Generate a heatmap with annotations for all cells, rounded to two decimal places
sns.heatmap(corr_matrix_cleaned, annot=True,  fmt=".2f",  cmap='coolwarm',  cbar=True, square=True)

# Add titles and labels for clarity
plt.title('Correlation Matrix of Numerical Variables')
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)

# Show plot
plt.tight_layout()  # Adjust layout to fit everything nicely
plt.show()

In [ ]:

# Object columns to analyze against the numerical one
object_cols = ['Constructor','Brand', 'Veh_Model', 'Veh_type', 'Version', 'Energy', 'Fuel_mode']
numeric_col = 'CO2_wltp'

# Set the overall figure size to accommodate the subplots
plt.figure(figsize=(15, 4 * len(object_cols)))

for i, obj_col in enumerate(object_cols):
    # Calculate the mean of 'CO2_wltp' for each category within the object variable
    df_non_nan = df_ptwltp[df_ptwltp[numeric_col].notna()]
    mean_co2_by_category = df_non_nan.groupby(obj_col)[numeric_col].mean().sort_values()

    # Get the top and bottom 7 categories
    top_categories = mean_co2_by_category.tail(7).index
    bottom_categories = mean_co2_by_category.head(7).index
    
    # Create two subplots side by side for the top and bottom 7 categories
    fig, axes = plt.subplots(1, 2, figsize=(15, 4), sharey=False)

    # Top 7 categories
    sns.boxplot(y=obj_col, x=numeric_col, data=df_non_nan[df_non_nan[obj_col].isin(top_categories)],
                ax=axes[0], order=top_categories)
    axes[0].set_title(f'Top 7 {obj_col} by Highest {numeric_col}')
    axes[0].tick_params(axis='y', rotation=0)

    # Bottom 7 categories
    sns.boxplot(y=obj_col, x=numeric_col, data=df_non_nan[df_non_nan[obj_col].isin(bottom_categories)],
                ax=axes[1], order=bottom_categories)
    axes[1].set_title(f'Bottom 7 {obj_col} by Lowest {numeric_col}')
    axes[1].tick_params(axis='y', rotation=0)

    # Display the subplots for the current object variable
    plt.tight_layout()
    plt.show()

In [ ]:
# Object columns to analyze against the numerical ones
object_cols = ['Veh_Category', 'Energy', 'Fuel_mode', 'Eco-innovation program']
numeric_col = 'CO2_wltp'

# Create a figure to contain all the subplots
fig, axes = plt.subplots(nrows=len(object_cols)+1, ncols=2, figsize=(20, 7 * (len(object_cols)+1)), gridspec_kw={'width_ratios': [1, 1]})

# Remove the last axes on the right as we will not use it
fig.delaxes(axes[-1, -1])

for i, obj_col in enumerate(object_cols):
    # Calculate means for each category within object column
    category_means = df_ptwltp.groupby(obj_col)[numeric_col].mean().sort_values()
    
    # Find the Top 7 categories by highest mean CO2_wltp
    top_categories = category_means.tail(7).index
    # Find the Bottom 7 categories by lowest mean CO2_wltp
    bottom_categories = category_means.head(7).index

    # Top 7 categories boxplot
    sns.boxplot(x=numeric_col, y=obj_col, data=df_ptwltp[df_ptwltp[obj_col].isin(top_categories)],
                ax=axes[i, 0], order=top_categories)
    axes[i, 0].set_title(f'Top 7 {obj_col} by Highest Average {numeric_col}')
    
    # Bottom 7 categories boxplot
    sns.boxplot(x=numeric_col, y=obj_col, data=df_ptwltp[df_ptwltp[obj_col].isin(bottom_categories)],
                ax=axes[i, 1], order=bottom_categories)
    axes[i, 1].set_title(f'Bottom 7 {obj_col} by Lowest Average {numeric_col}')
    
    # Set the rotation for x-tick labels
    axes[i, 0].tick_params(axis='y', rotation=0)
    axes[i, 1].tick_params(axis='y', rotation=0)

# Handle 'year' and 'Eco-innovation program' separately
# Year distribution for CO2_wltp
sns.boxplot(x='year', y=numeric_col, data=df_ptwltp, ax=axes[len(object_cols), 0])
axes[len(object_cols), 0].set_title(f'Distribution of {numeric_col} by Year')
axes[len(object_cols), 0].tick_params(axis='x', rotation=90)

# Eco-innovation program distribution for CO2_wltp
sns.boxplot(x='Eco-innovation program', y=numeric_col, data=df_ptwltp, ax=axes[len(object_cols), 1])
axes[len(object_cols), 1].set_title(f'Distribution of {numeric_col} by Eco-innovation Program')

# Adjust layout for better fit and to handle the titles and x labels
plt.tight_layout()

# Show the final plot
plt.show()
